<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">LLM Ops - Tracing, Evaluation, and Analysis</h1>

In this tutorial we will learn how to build, observe, and evaluate a LLM powered application. 

It has the following sections:

1. Understanding LLM powered applications
2. Building a LLM powered application
3. Observing the application using traces
4. Evaluating the application using LLM Evals
5. Exploring and and troubleshooting the application using UMAP projection and clustering


## Understanding LLM powered applications

Building software with LLMs, or any machine learning model, is [fundamentally different](https://karpathy.medium.com/software-2-0-a64152b37c35). Rather than compiling source code into binary to run a series of commands, we need to navigate datasets, embeddings, prompts, and parameter weights to generate consistent accurate results. LLM outputs are probabilistic and therefor don't produce the same predictable outcome every time.

<img src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/images/blog/5_steps_of_building_llm_app.png" />

There's a lot that can go into building an LLM application, but let's focus on the architecture. Below is a diagram of one possible architecture. In this diagram we see that the application is built around an LLM but that there are many other components that are needed to make the application work.

<img src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/images/blog/llm_app_architecture.png" />

The complexity that is involved in building an LLM application is why observability is so important. Observability is the ability to understand the internal state of a system by examining its inputs and outputs. Each step of the response generation process needs to be monitored, evaluated and tuned to provide the best possible experience. Not only that, certain tradeoffs might need to be made to optimize for speed, cost, or accuracy. In the context of LLM applications, we need to be able to understand the internal state of the system by examining telemetry data such as LLM Traces.


## Observing the application using traces

LLM Traces and Observability lets us understand the system from the outside, by letting us ask questions about that system without knowing its inner workings. Furthermore, it allows us to easily troubleshoot and handle novel problems (i.e. “unknown unknowns”), and helps us answer the question, “Why is this happening?”

To illustrate this, let's look at an example LLM application and inspect it's traces.

In [ ]:
!pip install -qq "arize-phoenix[experimental,llama-index]" "openai>=1" gcsfs

⚠️ This tutorial requires an OpenAI key to run. 

In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
import phoenix as px
from llama_index import set_global_handler

# Setup phoenix tracing
px.launch_app()
set_global_handler("arize_phoenix")

In [ ]:
import os
from getpass import getpass

import phoenix as px
from gcsfs import GCSFileSystem
from llama_index import (
    ServiceContext,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore
from llama_index.llms import OpenAI

file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),  # prevents unauthorized request to GCS
)

service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
)
index = load_index_from_storage(
    storage_context,
    service_context=service_context,
)
query_engine = index.as_query_engine()

Now that we have an application setup, let's take a look inside.

In [ ]:
import tqdm

queries = [
    "How can I query for a monitor's status using GraphQL?",
    "How do I delete a model?",
    "How much does an enterprise license of Arize cost?",
    "How do I log a prediction using the python SDK?",
]

for query in tqdm(queries[:30]):
    response = query_engine.query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")